PO/ER460 - Multi-criteria analysis course at Unicamp

Selecting and ordering elements of the swot matrix through a multicriteria and group decision-making approach

- Augusto Chebel Machado (231762)
- Émerson José Ferri
- Lucas Tramonte (182697)

# What does this code do?

- Using the PROMETHEE method, we rank each quadrant of the SWOT matrix (Opportunities, Threats, Weaknesses and Strengths);
- Each decision-maker generates their own ranking;
- After the individual ranking, using the Borda and Condorcet methods for group decisions, the rankings are generated for the group of decision-makers.

#  Libraries 

In [135]:
import pandas as pd
import numpy as np
import condorcet

# Functions 

## Defining functions for the PROMETHEE Method

In [136]:
def f_pref(df, criterio, tipo, func='usual', q=0, p=0):
  n_alt = len(df)
  m = np.zeros([n_alt, n_alt])
  for i in range(0, n_alt):
    for j in range(0, n_alt):
      if tipo == 'max':
        d = df[criterio][i] - df[criterio][j]
      elif tipo == 'min':
        d = df[criterio][j] - df[criterio][i]

      if func == 'usual':
        if d <= 0:
          m[i][j] = 0
        else:
          m[i][j] = 1

      if func == 'U':
        if d <= q:
          m[i][j] = 0
        else:
          m[i][j] = 1

      if func == 'níveis':
        if d <= 0:
          m[i][j] = 0
        elif q < d <= p:
          m[i][j] = 1/2
        else:
          m[i][j] = 1

      if func == 'linear':
        if d <= q:
          m[i][j] = 0
        elif q < d <= p:
          m[i][j] = (d-q)/(p-q)
        else:
          m[i][j] = 1

  return m

# flow calculation
def fluxo_pos(M):
  fluxo_pos = []
  for i in range(0, len(M)):
    phi = sum(M[i])/(len(M)-1)
    fluxo_pos.append(phi)
  return np.array(fluxo_pos)

def fluxo_neg(M):
  fluxo_neg = []
  for j in range(0, len(M)):
    phi = sum(M[...,j])/(len(M)-1)
    fluxo_neg.append(phi)
  return np.array(fluxo_neg)

def fluxo_liq(fluxo_pos, fluxo_neg):
  fluxo_liq = fluxo_pos - fluxo_neg
  return fluxo_liq

## Defining functions for the Condorcet Method

In [137]:
def voto(lista_df):
  votos = []
  for df in lista_df:
    voto_decisor = {}
    for i in range(0, len(df)):
      voto_decisor.update({df.index.values[i] : i+1})
    votos.append(voto_decisor)
  return votos

# Read data

### Opportunities

In [138]:
oport_a1 = pd.read_excel('Assets\Datasets\PO_ER460 - Opportunities.xlsx', sheet_name=0)
oport_a2 = pd.read_excel('Assets\Datasets\PO_ER460 - Opportunities.xlsx', sheet_name=1)
oport_a3 = pd.read_excel('Assets\Datasets\PO_ER460 - Opportunities.xlsx', sheet_name=2)
oport_a4 = pd.read_excel('Assets\Datasets\PO_ER460 - Opportunities.xlsx', sheet_name=3)
oport_a5 = pd.read_excel('Assets\Datasets\PO_ER460 - Opportunities.xlsx', sheet_name=4)
oport_a6 = pd.read_excel('Assets\Datasets\PO_ER460 - Opportunities.xlsx', sheet_name=5)
oport_a7 = pd.read_excel('Assets\Datasets\PO_ER460 - Opportunities.xlsx', sheet_name=6)
oport_a8 = pd.read_excel('Assets\Datasets\PO_ER460 - Opportunities.xlsx', sheet_name=7)
oport_a9 = pd.read_excel('Assets\Datasets\PO_ER460 - Opportunities.xlsx', sheet_name=8)
oport_a10 = pd.read_excel('Assets\Datasets\PO_ER460 - Opportunities.xlsx', sheet_name=9)
oport_a11 = pd.read_excel('Assets\Datasets\PO_ER460 - Opportunities.xlsx', sheet_name=10)

In [139]:
# making the alternatives like indexes
def indice(df):
  df = df.set_index('Alternativas/Critérios')
  df.index.names = [None]
  return df

oport_a1 = indice(oport_a1)
oport_a2 = indice(oport_a2)
oport_a3 = indice(oport_a3)
oport_a4 = indice(oport_a4)
oport_a5 = indice(oport_a5)
oport_a6 = indice(oport_a6)
oport_a7 = indice(oport_a7)
oport_a8 = indice(oport_a8)
oport_a9 = indice(oport_a9)
oport_a10 = indice(oport_a10)
oport_a11 = indice(oport_a11)

In [140]:
# creating a numerical scale and passing it to df
def escala(df):
  dic = {'Muito Alto':5, 'Alto':4 , 'Médio':3, 'Baixo':2, 'Muito Baixo':1}
  df['Probabilidade'] = df['Probabilidade'].map(dic)
  df['Custo'] = df['Custo'].map(dic)
  df['Benefício'] = df['Benefício'].map(dic)
  df['Visão'] = df['Visão'].map(dic)
  return df

oport_a1 = escala(oport_a1)
oport_a2 = escala(oport_a2)
oport_a3 = escala(oport_a3)
oport_a4 = escala(oport_a4)
oport_a5 = escala(oport_a5)
oport_a6 = escala(oport_a6)
oport_a7 = escala(oport_a7)
oport_a8 = escala(oport_a8)
oport_a9 = escala(oport_a9)
oport_a10 = escala(oport_a10)
oport_a11 = escala(oport_a11)

In [141]:
# creating function to calculate preference matrix M
def M_oport(df, w):
  m_fortes = f_pref(df, 'Pontos Fortes', 'linear', 'max', q=1, p=4)
  m_fracos = f_pref(df, 'Pontos Fracos', 'linear', 'min', q=1, p=4)
  m_prob = f_pref(df, 'Probabilidade', 'max')
  m_custo = f_pref(df, 'Custo', 'min')
  m_benef = f_pref(df, 'Benefício', 'max')
  m_visao = f_pref(df, 'Visão', 'max')

  matrizes = [m_fortes, m_fracos, m_prob, m_custo, m_benef, m_visao]

  M = 0
  for i in range(0, len(matrizes)):
    M = M + w[i]*matrizes[i]

  return M

# vector of weights
w_oport = [0.21, 0.09, 0.13, 0.11, 0.12, 0.34]

# calculation of preference matrices for each decision-maker (opportunities):
M_a1 = M_oport(oport_a1, w_oport)
M_a2 = M_oport(oport_a2, w_oport)
M_a3 = M_oport(oport_a3, w_oport)
M_a4 = M_oport(oport_a4, w_oport)
M_a5 = M_oport(oport_a5, w_oport)
M_a6 = M_oport(oport_a6, w_oport)
M_a7 = M_oport(oport_a7, w_oport)
M_a8 = M_oport(oport_a8, w_oport)
M_a9 = M_oport(oport_a9, w_oport)
M_a10 = M_oport(oport_a10, w_oport)
M_a11 = M_oport(oport_a11, w_oport)


C:\Users\lucas\AppData\Local\Temp\ipykernel_16368\2012082991.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  d = df[criterio][i] - df[criterio][j]
C:\Users\lucas\AppData\Local\Temp\ipykernel_16368\2012082991.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  d = df[criterio][j] - df[criterio][i]


In [142]:
# calculating the net flow
def fl_oport(M):
  fl = fluxo_liq(fluxo_pos(M), fluxo_neg(M))
  return fl

fl_a1 = fl_oport(M_a1)
fl_a2 = fl_oport(M_a2)
fl_a3 = fl_oport(M_a3)
fl_a4 = fl_oport(M_a4)
fl_a5 = fl_oport(M_a5)
fl_a6 = fl_oport(M_a6)
fl_a7 = fl_oport(M_a7)
fl_a8 = fl_oport(M_a8)
fl_a9 = fl_oport(M_a9)
fl_a10 = fl_oport(M_a10)
fl_a11 = fl_oport(M_a11)

In [143]:
# function for sorting
def ordenamento(decisor, df, fl):
  # generating a dictionary for ranking
  rank = {}
  for i in range(0, len(df)):
    rank.update({df.index.values[i] : fl[i]})

  tabela = pd.DataFrame(pd.Series(rank), columns=['Fluxo Líquido'])

  tabela = tabela.sort_values(by='Fluxo Líquido', ascending=False)

  ord = []
  for i in range(0, len(df)):
    ord.append(i+1)

  tabela[f'Rank D{decisor}'] = ord

  return tabela

In [144]:
print('Avaliador 1:')
print(ordenamento(1, oport_a1, fl_a1))
print(80*'-')
print('Avaliador 2:')
print(ordenamento(2, oport_a2, fl_a2))
print(80*'-')
print('Avaliador 3:')
print(ordenamento(3, oport_a3, fl_a3))
print(80*'-')
print('Avaliador 4:')
print(ordenamento(4, oport_a4, fl_a4))
print(80*'-')
print('Avaliador 5:')
print(ordenamento(5, oport_a5, fl_a5))
print(80*'-')
print('Avaliador 6:')
print(ordenamento(6, oport_a6, fl_a6))
print(80*'-')
print('Avaliador 7:')
print(ordenamento(7, oport_a7, fl_a7))
print(80*'-')
print('Avaliador 8:')
print(ordenamento(8, oport_a8, fl_a8))
print(80*'-')
print('Avaliador 9:')
print(ordenamento(9, oport_a9, fl_a9))
print(80*'-')
print('Avaliador 10:')
print(ordenamento(10, oport_a10, fl_a10))
print(80*'-')
print('Avaliador 11:')
print(ordenamento(11, oport_a11, fl_a11))
print(80*'-')

Avaliador 1:
                                                    Fluxo Líquido  Rank D1
Demanda regional por um novo serviço de atenção...       0.384286        1
Políticas públicas: financiamento (emendas) e i...       0.118571        2
Ambiente interno favorável à coordenação mútua           0.118571        3
Desenvolvimento de novas tecnologias da informação       0.004286        4
Prestígio social da Unicamp                             -0.007143        5
Nova lei de licitação 14.133/2021                       -0.072857        6
Legislação favorável a parcerias público-privada        -0.210000        7
Gestão de RH: aperfeiçoamento dos métodos de co...      -0.335714        8
--------------------------------------------------------------------------------
Avaliador 2:
                                                    Fluxo Líquido  Rank D2
Nova lei de licitação 14.133/2021                        0.384286        1
Demanda regional por um novo serviço de atenção...       0.278571   

In [145]:
# generating the tables for each decision-maker (opportunities)

ordem_oport_a1 = ordenamento(1, oport_a1, fl_a1)
ordem_oport_a2 = ordenamento(2, oport_a2, fl_a2)
ordem_oport_a3 = ordenamento(3, oport_a3, fl_a3)
ordem_oport_a4 = ordenamento(4, oport_a4, fl_a4)
ordem_oport_a5 = ordenamento(5, oport_a5, fl_a5)
ordem_oport_a6 = ordenamento(6, oport_a6, fl_a6)
ordem_oport_a7 = ordenamento(7, oport_a7, fl_a7)
ordem_oport_a8 = ordenamento(8, oport_a8, fl_a8)
ordem_oport_a9 = ordenamento(9, oport_a9, fl_a9)
ordem_oport_a10 = ordenamento(10, oport_a10, fl_a10)
ordem_oport_a11 = ordenamento(11, oport_a11, fl_a11)

# creating a dataframe with the rankings of all the decision-makers:
oportuni = pd.concat([ordem_oport_a1, ordem_oport_a2, ordem_oport_a3, ordem_oport_a4, ordem_oport_a5,ordem_oport_a6,ordem_oport_a7,ordem_oport_a8,ordem_oport_a9,ordem_oport_a10,ordem_oport_a11], axis=1)
oportuni
oportuni = oportuni.drop(['Fluxo Líquido'], axis=1)
oportuni

,Rank D1,Rank D2,Rank D3,Rank D4,Rank D5,Rank D6,Rank D7,Rank D8,Rank D9,Rank D10,Rank D11
Demanda regional por um novo serviço de atenção especializada à Saúde,1,2,1,4,2,4,2,6,7,1,1
Políticas públicas: financiamento (emendas) e integração Unicamp a novas políticas públicas de saúde,2,4,4,1,7,5,8,7,5,2,5
Ambiente interno favorável à coordenação mútua,3,5,2,3,4,8,7,3,3,5,8
Desenvolvimento de novas tecnologias da informação,4,8,3,6,3,6,6,4,8,4,6
Prestígio social da Unicamp,5,3,7,2,1,1,3,1,2,3,2
Nova lei de licitação 14.133/2021,6,1,5,5,8,3,4,8,6,7,3
Legislação favorável a parcerias público-privada,7,7,6,7,6,7,1,5,4,6,4
Gestão de RH: aperfeiçoamento dos métodos de controle de ponto e implantação de projeto de perícias médicas,8,6,8,8,5,2,5,2,1,8,7


# Group Decision (Borda)

In [146]:
# creating a scale for the border
borda_oport = {}
for i in range(0, len(oportuni)):
  borda_oport.update({i+1: (len(oportuni)-i)})

peso_dirigente = 1.2
borda_oport_dirig = {}
for i in range(0, len(oportuni)):
  borda_oport_dirig.update({i+1: (len(oportuni)-i)*peso_dirigente})

# replacing the sort values with the Edge scale value
# Ex: (1,2,3) will become (3,2,1)
for i in oportuni.columns:
  if i == oportuni.columns[0]: # se o decisor for a dirigente (peso maior)
    oportuni[i] = oportuni[i].map(borda_oport_dirig)
  else:
    oportuni[i] = oportuni[i].map(borda_oport)

# adding up the values of each alternative
oportuni['Valor'] = oportuni.sum(axis=1)

oport_final = pd.DataFrame(oportuni['Valor'])
oport_final = oport_final.sort_values(by='Valor', ascending=False)

ord = []
for i in range(0, len(oport_final)):
  ord.append(f'{i+1}°')

oport_final['Ordem Final (Borda)'] = ord
oport_final

,Valor,Ordem Final (Borda)
Prestígio social da Unicamp,69.8,1°
Demanda regional por um novo serviço de atenção especializada à Saúde,69.6,2°
Políticas públicas: financiamento (emendas) e integração Unicamp a novas políticas públicas de saúde,50.4,3°
Ambiente interno favorável à coordenação mútua,49.2,4°
Nova lei de licitação 14.133/2021,43.6,5°
Desenvolvimento de novas tecnologias da informação,42.0,6°
Legislação favorável a parcerias público-privada,39.4,7°
Gestão de RH: aperfeiçoamento dos métodos de controle de ponto e implantação de projeto de perícias médicas,39.2,8°


Group decision-making (Condorcet)

In [147]:
ordem_oport_a1 = ordenamento(1, oport_a1, fl_a1)
ordem_oport_a2 = ordenamento(2, oport_a2, fl_a2)
ordem_oport_a3 = ordenamento(3, oport_a3, fl_a3)
ordem_oport_a4 = ordenamento(4, oport_a4, fl_a4)
ordem_oport_a5 = ordenamento(5, oport_a5, fl_a5)
ordem_oport_a6 = ordenamento(6, oport_a6, fl_a6)
ordem_oport_a7 = ordenamento(7, oport_a7, fl_a7)
ordem_oport_a8 = ordenamento(8, oport_a8, fl_a8)
ordem_oport_a9 = ordenamento(9, oport_a9, fl_a9)
ordem_oport_a10 = ordenamento(10, oport_a10, fl_a10)
ordem_oport_a11 = ordenamento(11, oport_a11, fl_a11)

candidatos = ordem_oport_a1.index.values
candidatos = list(candidatos)

lista_oport = [ordem_oport_a1, ordem_oport_a2, ordem_oport_a3, ordem_oport_a4, ordem_oport_a5, ordem_oport_a6,ordem_oport_a7,ordem_oport_a8,ordem_oport_a9,ordem_oport_a10,ordem_oport_a11]
votos = voto(lista_oport)

# using condorcet library functions to define sorting
evaluator = condorcet.CondorcetEvaluator(candidates=candidatos, votes=votos)
winners, rest_of_table = evaluator.get_n_winners(len(ordem_oport_a1))

oport_condorcet = pd.DataFrame(index=winners)

ord = []
for i in range(0, len(oport_condorcet)):
  ord.append(f'{i+1}°')
oport_condorcet['Ordem Final (Condorcet)'] = ord

oport_condorcet

,Ordem Final (Condorcet)
Demanda regional por um novo serviço de atenção especializada à Saúde,1°
Prestígio social da Unicamp,2°
Políticas públicas: financiamento (emendas) e integração Unicamp a novas políticas públicas de saúde,3°
Ambiente interno favorável à coordenação mútua,4°
Nova lei de licitação 14.133/2021,5°
Desenvolvimento de novas tecnologias da informação,6°
Legislação favorável a parcerias público-privada,7°
Gestão de RH: aperfeiçoamento dos métodos de controle de ponto e implantação de projeto de perícias médicas,8°


# Threats

In [148]:
# Importing problem data (threats)

ameaça_a1 = pd.read_excel('Assets\Datasets\PO_ER460 - Threats.xlsx', sheet_name=0)
ameaça_a2 = pd.read_excel('Assets\Datasets\PO_ER460 - Threats.xlsx', sheet_name=1)
ameaça_a3 = pd.read_excel('Assets\Datasets\PO_ER460 - Threats.xlsx', sheet_name=2)
ameaça_a4 = pd.read_excel('Assets\Datasets\PO_ER460 - Threats.xlsx', sheet_name=3)
ameaça_a5 = pd.read_excel('Assets\Datasets\PO_ER460 - Threats.xlsx', sheet_name=4)
ameaça_a6 = pd.read_excel('Assets\Datasets\PO_ER460 - Threats.xlsx', sheet_name=5)
ameaça_a7 = pd.read_excel('Assets\Datasets\PO_ER460 - Threats.xlsx', sheet_name=6)
ameaça_a8 = pd.read_excel('Assets\Datasets\PO_ER460 - Threats.xlsx', sheet_name=7)
ameaça_a9 = pd.read_excel('Assets\Datasets\PO_ER460 - Threats.xlsx', sheet_name=8)
ameaça_a10 = pd.read_excel('Assets\Datasets\PO_ER460 - Threats.xlsx', sheet_name=9)
ameaça_a11 = pd.read_excel('Assets\Datasets\PO_ER460 - Threats.xlsx', sheet_name=10)

In [149]:
# making the alternatives like indexes
def indice(df):
  df = df.set_index('Alternativas/Critérios')
  df.index.names = [None]
  return df

ameaça_a1 = indice(ameaça_a1)
ameaça_a2 = indice(ameaça_a2)
ameaça_a3 = indice(ameaça_a3)
ameaça_a4 = indice(ameaça_a4)
ameaça_a5 = indice(ameaça_a5)
ameaça_a6 = indice(ameaça_a6)
ameaça_a7 = indice(ameaça_a7)
ameaça_a8 = indice(ameaça_a8)
ameaça_a9 = indice(ameaça_a9)
ameaça_a10 = indice(ameaça_a10)
ameaça_a11 = indice(ameaça_a11)

In [150]:
# criando uma escala numérica e passando para o df
def escala(df):
  dic = {'Muito Alto':5, 'Alto':4 , 'Médio':3, 'Baixo':2, 'Muito Baixo':1}
  df['Probabilidade'] = df['Probabilidade'].map(dic)
  df['Custo'] = df['Custo'].map(dic)
  df['Impacto'] = df['Impacto'].map(dic)
  df['Visão'] = df['Visão'].map(dic)
  return df

ameaça_a1 = escala(ameaça_a1)
ameaça_a2 = escala(ameaça_a2)
ameaça_a3 = escala(ameaça_a3)
ameaça_a4 = escala(ameaça_a4)
ameaça_a5 = escala(ameaça_a5)
ameaça_a6 = escala(ameaça_a6)
ameaça_a7 = escala(ameaça_a7)
ameaça_a8 = escala(ameaça_a8)
ameaça_a9 = escala(ameaça_a9)
ameaça_a10 = escala(ameaça_a10)
ameaça_a11 = escala(ameaça_a11)

In [151]:
# creating function to calculate preference matrix M
def M_ameaca(df, w):
  m_fracos = f_pref(df, 'Pontos Fracos', 'min', 'linear', q=1, p=4)
  m_fortes = f_pref(df, 'Pontos Fortes', 'max', 'linear', q=1, p=4)
  m_prob = f_pref(df, 'Probabilidade', 'max')
  m_custo = f_pref(df, 'Custo', 'min')
  m_impac = f_pref(df, 'Impacto', 'max')
  m_visao = f_pref(df, 'Visão', 'max')

  matrizes = [m_fortes, m_fracos, m_prob, m_custo, m_impac, m_visao]

  M = 0
  for i in range(0, len(matrizes)):
    M = M + w[i]*matrizes[i]

  return M

# vector of weights
w_ameaça = [0.18, 0.12, 0.18, 0.08, 0.12, 0.32]

# calculation of preference matrices for each decision-maker (threats):
M_a1 = M_ameaca(ameaça_a1, w_ameaça)
M_a2 = M_ameaca(ameaça_a2, w_ameaça)
M_a3 = M_ameaca(ameaça_a3, w_ameaça)
M_a4 = M_ameaca(ameaça_a4, w_ameaça)
M_a5 = M_ameaca(ameaça_a5, w_ameaça)
M_a6 = M_ameaca(ameaça_a6, w_ameaça)
M_a7 = M_ameaca(ameaça_a7, w_ameaça)
M_a8 = M_ameaca(ameaça_a8, w_ameaça)
M_a9 = M_ameaca(ameaça_a9, w_ameaça)
M_a10 = M_ameaca(ameaça_a10, w_ameaça)
M_a11 = M_ameaca(ameaça_a11, w_ameaça)

C:\Users\lucas\AppData\Local\Temp\ipykernel_16368\2012082991.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  d = df[criterio][j] - df[criterio][i]
C:\Users\lucas\AppData\Local\Temp\ipykernel_16368\2012082991.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  d = df[criterio][i] - df[criterio][j]


In [152]:
# calculating the net flow
def fl_ameaca(M):
  fl = fluxo_liq(fluxo_pos(M), fluxo_neg(M))
  return fl

fl_a1 = fl_ameaca(M_a1)
fl_a2 = fl_ameaca(M_a2)
fl_a3 = fl_ameaca(M_a3)
fl_a4 = fl_ameaca(M_a4)
fl_a5 = fl_ameaca(M_a5)
fl_a6 = fl_ameaca(M_a6)
fl_a7 = fl_ameaca(M_a7)
fl_a8 = fl_ameaca(M_a8)
fl_a9 = fl_ameaca(M_a9)
fl_a10 = fl_ameaca(M_a10)
fl_a11 = fl_ameaca(M_a11)

In [153]:
# function for sorting
def ordenamento(decisor, df, fl):
  # generating a dictionary for ranking
  rank = {}
  for i in range(0, len(df)):
    rank.update({df.index.values[i] : fl[i]})

  tabela = pd.DataFrame(pd.Series(rank), columns=['Fluxo Líquido'])

  tabela = tabela.sort_values(by='Fluxo Líquido', ascending=False)

  ord = []
  for i in range(0, len(df)):
    ord.append(i+1)

  tabela[f'Rank D{decisor}'] = ord

  return tabela

In [154]:
print('Avaliador 1:')
print(ordenamento(1, ameaça_a1, fl_a1))
print(80*'-')
print('Avaliador 2:')
print(ordenamento(2, ameaça_a2, fl_a2))
print(80*'-')
print('Avaliador 3:')
print(ordenamento(3, ameaça_a3, fl_a3))
print(80*'-')
print('Avaliador 4:')
print(ordenamento(4, ameaça_a4, fl_a4))
print(80*'-')
print('Avaliador 5:')
print(ordenamento(5, ameaça_a5, fl_a5))
print(80*'-')
print('Avaliador 6:')
print(ordenamento(6, ameaça_a6, fl_a6))
print(80*'-')
print('Avaliador 7:')
print(ordenamento(7, ameaça_a7, fl_a7))
print(80*'-')
print('Avaliador 8:')
print(ordenamento(8, ameaça_a8, fl_a8))
print(80*'-')
print('Avaliador 9:')
print(ordenamento(9, ameaça_a9, fl_a9))
print(80*'-')
print('Avaliador 10:')
print(ordenamento(10, ameaça_a10, fl_a10))
print(80*'-')
print('Avaliador 11:')
print(ordenamento(11, ameaça_a11, fl_a11))
print(80*'-')

Avaliador 1:
                                                    Fluxo Líquido  Rank D1
Distorções existentes nos serviços de saúde cau...       0.235789        1
Subfinanciamento do SUS                                  0.230526        2
Administração Central insensível às demandas da...       0.204211        3
Grau de independência de cada serviço de saúde ...       0.164211        4
Instabilidade e inadequação dos recursos financ...       0.153684        5
Ausência de interlocução do DGRH com a DEAS em ...       0.113684        6
Sistemas informatizados da universidade e ausên...       0.082105        7
Resistência das unidades próprias de Saúde à ce...       0.078947        8
Fragilidades relacionadas ao modelo de gestão d...       0.063158        9
Instabilidade do financiamento devido às mudanç...       0.049474       10
Ponto eletrônico pode gerar processos trabalhis...       0.037895       11
Readequações necessárias para o cumprimento da ...       0.030526       12
Insuficiênci

In [155]:
# generating the tables for each decision-maker (threats)
ordem_ameaça_a1 = ordenamento(1, ameaça_a1, fl_a1)
ordem_ameaça_a2 = ordenamento(2, ameaça_a2, fl_a2)
ordem_ameaça_a3 = ordenamento(3, ameaça_a3, fl_a3)
ordem_ameaça_a4 = ordenamento(4, ameaça_a4, fl_a4)
ordem_ameaça_a5 = ordenamento(5, ameaça_a5, fl_a5)
ordem_ameaça_a6 = ordenamento(6, ameaça_a6, fl_a6)
ordem_ameaça_a7 = ordenamento(7, ameaça_a7, fl_a7)
ordem_ameaça_a8 = ordenamento(8, ameaça_a8, fl_a8)
ordem_ameaça_a9 = ordenamento(9, ameaça_a9, fl_a9)
ordem_ameaça_a10 = ordenamento(10, ameaça_a10, fl_a10)
ordem_ameaça_a11 = ordenamento(11, ameaça_a11, fl_a11)

# creating a dataframe with the rankings of all the decision-makers:
ameaça = pd.concat([ordem_ameaça_a1, ordem_ameaça_a2, ordem_ameaça_a3, ordem_ameaça_a4, ordem_ameaça_a5, ordem_ameaça_a6, ordem_ameaça_a7, ordem_ameaça_a8, ordem_ameaça_a9, ordem_ameaça_a10, ordem_ameaça_a11], axis=1)
ameaça = ameaça.drop(['Fluxo Líquido'], axis=1)
ameaça

,Rank D1,Rank D2,Rank D3,Rank D4,Rank D5,Rank D6,Rank D7,Rank D8,Rank D9,Rank D10,Rank D11
Distorções existentes nos serviços de saúde causados pelo uso inadequado do trabalho remoto,1,15,19,18,19,8,20,19,20,20,13
Subfinanciamento do SUS,2,19,6,10,7,12,13,2,11,5,1
Administração Central insensível às demandas da área de saúde,3,6,9,2,15,15,10,9,3,9,10
Grau de independência de cada serviço de saúde vinculado ao DEAS em relação aos organismos gestores no SUS e demais forças políticas,4,2,2,7,1,6,4,13,10,7,5
Instabilidade e inadequação dos recursos financeiros dos convênios,5,8,5,11,8,16,5,4,8,3,3
"Ausência de interlocução do DGRH com a DEAS em relação à gestão de pessoas, reposição de quadro de pessoal, etc.",6,3,18,17,13,4,2,18,16,11,18
Sistemas informatizados da universidade e ausência de atualização,7,7,11,15,16,1,6,12,19,16,19
Resistência das unidades próprias de Saúde à centralização de gestão e processos na DEAS,8,1,1,5,4,10,11,11,2,10,6
"Fragilidades relacionadas ao modelo de gestão do SUS: burocratização dos processos, excesso de normatizações, grande número de instâncias",9,16,10,1,3,11,15,6,1,6,15
Instabilidade do financiamento devido às mudanças no governo,10,12,4,3,10,18,3,3,6,2,2


## Group Decision (Borda)

In [156]:
# creating a scale for the border
borda_ameaça = {}
for i in range(0, len(ameaça)):
  borda_ameaça.update({i+1: (len(ameaça)-i)})

peso_dirigente = 1.2
borda_ameaça_dirig = {}
for i in range(0, len(ameaça)):
  borda_ameaça_dirig.update({i+1: (len(ameaça)-i)*peso_dirigente})

# replacing the sort values with the Edge scale value
# Ex: (1,2,3) will become (3,2,1)
for i in ameaça.columns:
  if i == ameaça.columns[0]: # if the decision-maker is the manager (greater weight)
    ameaça[i] = ameaça[i].map(borda_ameaça_dirig)
  else:
    ameaça[i] = ameaça[i].map(borda_ameaça)

# adding up the values of each alternative
ameaça['Valor'] = ameaça.sum(axis=1)
# threat

ameaça_final = pd.DataFrame(ameaça['Valor'])
ameaça_final = ameaça_final.sort_values(by='Valor', ascending=False)

ord = []
for i in range(0, len(ameaça_final)):
  ord.append(f'{i+1}°')

ameaça_final['Ordem Final (Borda)'] = ord
ameaça_final

,Valor,Ordem Final (Borda)
Grau de independência de cada serviço de saúde vinculado ao DEAS em relação aos organismos gestores no SUS e demais forças políticas,173.4,1°
Resistência das unidades próprias de Saúde à centralização de gestão e processos na DEAS,164.6,2°
Instabilidade do financiamento devido às mudanças no governo,160.2,3°
Instabilidade e inadequação dos recursos financeiros dos convênios,158.2,4°
Insuficiência do financiamento das unidades convencionadas à Secretaria do Estado de São Paulo,157.6,5°
Subfinanciamento do SUS,146.8,6°
Administração Central insensível às demandas da área de saúde,143.6,7°
"Fragilidades relacionadas ao modelo de gestão do SUS: burocratização dos processos, excesso de normatizações, grande número de instâncias",140.4,8°
Fragilidades relacionadas ao modelo de atenção à Saúde do SUS: fragmentação,135.0,9°
Fragilidade de políticas específicas do Estado voltada aos serviços universitários,122.4,10°


## Group Decision (Condorcet):

In [157]:
ordem_ameaça_a1 = ordenamento(1, ameaça_a1, fl_a1)
ordem_ameaça_a2 = ordenamento(2, ameaça_a2, fl_a2)
ordem_ameaça_a3 = ordenamento(3, ameaça_a3, fl_a3)
ordem_ameaça_a4 = ordenamento(4, ameaça_a4, fl_a4)
ordem_ameaça_a5 = ordenamento(5, ameaça_a5, fl_a5)
ordem_ameaça_a6 = ordenamento(6, ameaça_a6, fl_a6)
ordem_ameaça_a7 = ordenamento(7, ameaça_a7, fl_a7)
ordem_ameaça_a8 = ordenamento(8, ameaça_a8, fl_a8)
ordem_ameaça_a9 = ordenamento(9, ameaça_a9, fl_a9)
ordem_ameaça_a10 = ordenamento(10, ameaça_a10, fl_a10)
ordem_ameaça_a11 = ordenamento(11, ameaça_a11, fl_a11)

candidatos = ordem_ameaça_a1.index.values
candidatos = list(candidatos)

lista_ameaça = [ordem_ameaça_a1, ordem_ameaça_a2, ordem_ameaça_a3, ordem_ameaça_a4, ordem_ameaça_a5, ordem_ameaça_a6, ordem_ameaça_a7, ordem_ameaça_a8, ordem_ameaça_a9, ordem_ameaça_a10, ordem_ameaça_a11]
votos = voto(lista_ameaça)

# using condorcet library functions to define sorting
evaluator = condorcet.CondorcetEvaluator(candidates=candidatos, votes=votos)
winners, rest_of_table = evaluator.get_n_winners(len(ordem_ameaça_a1))

ameaça_condorcet = pd.DataFrame(index=winners)

ord = []
for i in range(0, len(ameaça_condorcet)):
  ord.append(f'{i+1}°')
ameaça_condorcet['Ordem Final (Condorcet)'] = ord

ameaça_condorcet

,Ordem Final (Condorcet)


# Weak points

In [158]:
# Importing the problem data (weak points)

fraco_a1 = pd.read_excel('Assets\Datasets\PO_ER460 - Weak points.xlsx', sheet_name=0)
fraco_a2 = pd.read_excel('Assets\Datasets\PO_ER460 - Weak points.xlsx', sheet_name=1)
fraco_a3 = pd.read_excel('Assets\Datasets\PO_ER460 - Weak points.xlsx', sheet_name=2)
fraco_a4 = pd.read_excel('Assets\Datasets\PO_ER460 - Weak points.xlsx', sheet_name=3)
fraco_a5 = pd.read_excel('Assets\Datasets\PO_ER460 - Weak points.xlsx', sheet_name=4)
fraco_a6 = pd.read_excel('Assets\Datasets\PO_ER460 - Weak points.xlsx', sheet_name=5)
fraco_a7 = pd.read_excel('Assets\Datasets\PO_ER460 - Weak points.xlsx', sheet_name=6)
fraco_a8 = pd.read_excel('Assets\Datasets\PO_ER460 - Weak points.xlsx', sheet_name=7)
fraco_a9 = pd.read_excel('Assets\Datasets\PO_ER460 - Weak points.xlsx', sheet_name=8)
fraco_a10 = pd.read_excel('Assets\Datasets\PO_ER460 - Weak points.xlsx', sheet_name=9)
fraco_a11 = pd.read_excel('Assets\Datasets\PO_ER460 - Weak points.xlsx', sheet_name=10)

In [159]:
# making the alternatives like indexes
def indice(df):
  df = df.set_index('Alternativas/Critérios')
  df.index.names = [None]
  return df

fraco_a1 = indice(fraco_a1)
fraco_a2 = indice(fraco_a2)
fraco_a3 = indice(fraco_a3)
fraco_a4 = indice(fraco_a4)
fraco_a5 = indice(fraco_a5)
fraco_a6 = indice(fraco_a6)
fraco_a7 = indice(fraco_a7)
fraco_a8 = indice(fraco_a8)
fraco_a9 = indice(fraco_a9)
fraco_a10 = indice(fraco_a10)
fraco_a11 = indice(fraco_a11)

In [160]:
# creating a numerical scale and passing it to df
def escala(df):
  dic = {'Muito Alto':5, 'Alto':4 , 'Médio':3, 'Baixo':2, 'Muito Baixo':1}
  df['Custo'] = df['Custo'].map(dic)
  df['Retorno'] = df['Retorno'].map(dic)
  df['Missão'] = df['Missão'].map(dic)
  df['Visão'] = df['Visão'].map(dic)
  return df

fraco_a1 = escala(fraco_a1)
fraco_a2 = escala(fraco_a2)
fraco_a3 = escala(fraco_a3)
fraco_a4 = escala(fraco_a4)
fraco_a5 = escala(fraco_a5)
fraco_a6 = escala(fraco_a6)
fraco_a7 = escala(fraco_a7)
fraco_a8 = escala(fraco_a8)
fraco_a9 = escala(fraco_a9)
fraco_a10 = escala(fraco_a10)
fraco_a11 = escala(fraco_a11)

In [161]:
# creating a numerical scale and passing it to df
def M_fraco(df, w):
  m_pot_ame = f_pref(df, 'Pot. Ameaças', 'max', 'linear', q=1, p=4)
  m_dif_oport = f_pref(df, 'Dif. Oport.', 'max', 'linear', q=1, p=4)
  m_custo = f_pref(df, 'Custo', 'min')
  m_ret = f_pref(df, 'Retorno', 'max')
  m_missao = f_pref(df, 'Missão', 'max')
  m_visao = f_pref(df, 'Visão', 'max')

  matrizes = [m_pot_ame, m_dif_oport, m_custo, m_ret, m_missao, m_visao]

  M = 0
  for i in range(0, len(matrizes)):
    M = M + w[i]*matrizes[i]

  return M

# vector of weights
w_fraco = [0.09, 0.06, 0.12, 0.10, 0.23, 0.40]

# calculating the preference matrices for each decision-maker (weaknesses):
M_a1 = M_fraco(fraco_a1, w_fraco)
M_a2 = M_fraco(fraco_a2, w_fraco)
M_a3 = M_fraco(fraco_a3, w_fraco)
M_a4 = M_fraco(fraco_a4, w_fraco)
M_a5 = M_fraco(fraco_a5, w_fraco)
M_a6 = M_fraco(fraco_a6, w_fraco)
M_a7 = M_fraco(fraco_a7, w_fraco)
M_a8 = M_fraco(fraco_a8, w_fraco)
M_a9 = M_fraco(fraco_a9, w_fraco)
M_a10 = M_fraco(fraco_a10, w_fraco)
M_a11 = M_fraco(fraco_a11, w_fraco)

C:\Users\lucas\AppData\Local\Temp\ipykernel_16368\2012082991.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  d = df[criterio][i] - df[criterio][j]
C:\Users\lucas\AppData\Local\Temp\ipykernel_16368\2012082991.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  d = df[criterio][j] - df[criterio][i]


In [162]:
# calculating the net flow
def fl_fraco(M):
  fl = fluxo_liq(fluxo_pos(M), fluxo_neg(M))
  return fl

fl_a1 = fl_fraco(M_a1)
fl_a2 = fl_fraco(M_a2)
fl_a3 = fl_fraco(M_a3)
fl_a4 = fl_fraco(M_a4)
fl_a5 = fl_fraco(M_a5)
fl_a6 = fl_fraco(M_a6)
fl_a7 = fl_fraco(M_a7)
fl_a8 = fl_fraco(M_a8)
fl_a9 = fl_fraco(M_a9)
fl_a10 = fl_fraco(M_a10)
fl_a11 = fl_fraco(M_a11)

In [163]:
# function for sorting
def ordenamento(decisor, df, fl):
  # generating a dictionary for ranking
  rank = {}
  for i in range(0, len(df)):
    rank.update({df.index.values[i] : fl[i]})

  tabela = pd.DataFrame(pd.Series(rank), columns=['Fluxo Líquido'])

  tabela = tabela.sort_values(by='Fluxo Líquido', ascending=False)

  ord = []
  for i in range(0, len(df)):
    ord.append(i+1)

  tabela[f'Rank D{decisor}'] = ord

  return tabela

In [164]:
print('Avaliador 1:')
print(ordenamento(1, fraco_a1, fl_a1))
print(80*'-')
print('Avaliador 2:')
print(ordenamento(2, fraco_a2, fl_a2))
print(80*'-')
print('Avaliador 3:')
print(ordenamento(3, fraco_a3, fl_a3))
print(80*'-')
print('Avaliador 4:')
print(ordenamento(4, fraco_a4, fl_a4))
print(80*'-')
print('Avaliador 5:')
print(ordenamento(5, fraco_a5, fl_a5))
print(80*'-')
print('Avaliador 6:')
print(ordenamento(6, fraco_a6, fl_a6))
print(80*'-')
print('Avaliador 7:')
print(ordenamento(7, fraco_a7, fl_a7))
print(80*'-')
print('Avaliador 8:')
print(ordenamento(8, fraco_a8, fl_a8))
print(80*'-')
print('Avaliador 9:')
print(ordenamento(9, fraco_a9, fl_a9))
print(80*'-')
print('Avaliador 10:')
print(ordenamento(10, fraco_a10, fl_a10))
print(80*'-')
print('Avaliador 11:')
print(ordenamento(11, fraco_a11, fl_a11))
print(80*'-')

Avaliador 1:
                                                    Fluxo Líquido  Rank D1
Efetivação do papel institucional (não reconhec...       0.513333        1
Área física inadequada                                   0.460000        2
Fragilidade da equipe nos conhecimentos “relaci...       0.360833        3
Falta de autonomia financeira interna                    0.348333        4
Predominância de ações emergenciais em detrimen...       0.340833        5
Ausência de ferramentas gerenciais de diagnósti...       0.070000        6
Falta de integração dos processos de trabalho (...      -0.010000        7
Equipe reduzida                                         -0.042500        8
Ausência de integração entre pesquisa, ensino e...      -0.163333        9
O Conselho Executivo não participa das discussõ...      -0.345833       10
Falta organização e diretrizes para captação de...      -0.475833       11
Inexistência da centralização de compras na Áre...      -0.504167       12
A gestão vin

In [165]:
# generating the tables for each decision-maker (weaknesses)
ordem_fraco_a1 = ordenamento(1, fraco_a1, fl_a1)
ordem_fraco_a2 = ordenamento(2, fraco_a2, fl_a2)
ordem_fraco_a3 = ordenamento(3, fraco_a3, fl_a3)
ordem_fraco_a4 = ordenamento(4, fraco_a4, fl_a4)
ordem_fraco_a5 = ordenamento(5, fraco_a5, fl_a5)
ordem_fraco_a6 = ordenamento(6, fraco_a6, fl_a6)
ordem_fraco_a7 = ordenamento(7, fraco_a7, fl_a7)
ordem_fraco_a8 = ordenamento(8, fraco_a8, fl_a8)
ordem_fraco_a9 = ordenamento(9, fraco_a9, fl_a9)
ordem_fraco_a10 = ordenamento(10, fraco_a10, fl_a10)
ordem_fraco_a11 = ordenamento(11, fraco_a11, fl_a11)

# creating a dataframe with the rankings of all the decision-makers:
pont_fracos = pd.concat([ordem_fraco_a1, ordem_fraco_a2, ordem_fraco_a3, ordem_fraco_a4, ordem_fraco_a5, ordem_fraco_a6, ordem_fraco_a7, ordem_fraco_a8, ordem_fraco_a9, ordem_fraco_a10, ordem_fraco_a11], axis=1)
pont_fracos = pont_fracos.drop(['Fluxo Líquido'], axis=1)
pont_fracos

,Rank D1,Rank D2,Rank D3,Rank D4,Rank D5,Rank D6,Rank D7,Rank D8,Rank D9,Rank D10,Rank D11
Efetivação do papel institucional (não reconhecimento da missão/DEAS - interna e externa),1,1,2,11,4,2,2,4,9,1,6
Área física inadequada,2,13,12,13,11,5,13,13,13,8,13
Fragilidade da equipe nos conhecimentos “relacionados” ao SUS,3,10,6,8,8,6,9,3,3,3,12
Falta de autonomia financeira interna,4,5,11,3,3,12,10,9,12,5,7
Predominância de ações emergenciais em detrimento das ações estratégicas,5,11,7,4,5,1,6,6,2,4,8
Ausência de ferramentas gerenciais de diagnóstico e operação (indicadores),6,3,5,6,6,8,3,1,6,11,4
"Falta de integração dos processos de trabalho (reunião de equipe, desarticulação, troca de experiências, comunicação ineficiente)",7,6,1,5,13,3,5,5,1,10,2
Equipe reduzida,8,7,10,2,1,9,4,2,11,9,11
"Ausência de integração entre pesquisa, ensino e assistência",9,8,4,10,10,7,11,11,5,7,1
O Conselho Executivo não participa das discussões e resoluções de problemas (só informação na reunião),10,2,3,1,9,4,7,7,4,2,3


## Group Decision (Borda)

In [166]:
# creating a scale for the border
borda_fraco = {}
for i in range(0, len(pont_fracos)):
  borda_fraco.update({i+1: (len(pont_fracos)-i)})

peso_dirigente = 1.2
borda_fraco_dirig = {}
for i in range(0, len(pont_fracos)):
  borda_fraco_dirig.update({i+1: (len(pont_fracos)-i)*peso_dirigente})

# replacing the sort values with the Edge scale value
# Ex: (1,2,3) will become (3,2,1)
for i in pont_fracos.columns:
  if i == pont_fracos.columns[0]: # if the decision-maker is the manager (greater weight)
    pont_fracos[i] = pont_fracos[i].map(borda_fraco_dirig)
  else:
    pont_fracos[i] = pont_fracos[i].map(borda_fraco)

# adding up the values of each alternative
pont_fracos['Valor'] = pont_fracos.sum(axis=1)

pont_fracos_final = pd.DataFrame(pont_fracos['Valor'])
pont_fracos_final = pont_fracos_final.sort_values(by='Valor', ascending=False)

ord = []
for i in range(0, len(pont_fracos_final)):
  ord.append(f'{i+1}°')

pont_fracos_final['Ordem Final (Borda)'] = ord
pont_fracos_final

,Valor,Ordem Final (Borda)
Efetivação do papel institucional (não reconhecimento da missão/DEAS - interna e externa),113.6,1°
O Conselho Executivo não participa das discussões e resoluções de problemas (só informação na reunião),102.8,2°
"Falta de integração dos processos de trabalho (reunião de equipe, desarticulação, troca de experiências, comunicação ineficiente)",97.4,3°
Predominância de ações emergenciais em detrimento das ações estratégicas,96.8,4°
Ausência de ferramentas gerenciais de diagnóstico e operação (indicadores),96.6,5°
Fragilidade da equipe nos conhecimentos “relacionados” ao SUS,85.2,6°
Equipe reduzida,81.2,7°
Falta de autonomia financeira interna,75.0,8°
"Ausência de integração entre pesquisa, ensino e assistência",72.0,9°
Falta organização e diretrizes para captação de recursos de emendas,57.6,10°


## Group Decision (Condorcet):

In [167]:
ordem_fraco_a1 = ordenamento(1, fraco_a1, fl_a1)
ordem_fraco_a2 = ordenamento(2, fraco_a2, fl_a2)
ordem_fraco_a3 = ordenamento(3, fraco_a3, fl_a3)
ordem_fraco_a4 = ordenamento(4, fraco_a4, fl_a4)
ordem_fraco_a5 = ordenamento(5, fraco_a5, fl_a5)
ordem_fraco_a6 = ordenamento(6, fraco_a6, fl_a6)
ordem_fraco_a7 = ordenamento(7, fraco_a7, fl_a7)
ordem_fraco_a8 = ordenamento(8, fraco_a8, fl_a8)
ordem_fraco_a9 = ordenamento(9, fraco_a9, fl_a9)
ordem_fraco_a10 = ordenamento(10, fraco_a10, fl_a10)
ordem_fraco_a11 = ordenamento(11, fraco_a11, fl_a11)

candidatos = ordem_fraco_a1.index.values
candidatos = list(candidatos)

lista_fraco = [ordem_fraco_a1, ordem_fraco_a2, ordem_fraco_a3, ordem_fraco_a4, ordem_fraco_a5, ordem_fraco_a6, ordem_fraco_a7, ordem_fraco_a8, ordem_fraco_a9, ordem_fraco_a10, ordem_fraco_a11]
votos = voto(lista_fraco)

# using condorcet library functions to define sorting
evaluator = condorcet.CondorcetEvaluator(candidates=candidatos, votes=votos)
winners, rest_of_table = evaluator.get_n_winners(len(ordem_ameaça_a1))

fraco_condorcet = pd.DataFrame(index=winners)

ord = []
for i in range(0, len(fraco_condorcet)):
  ord.append(f'{i+1}°')
fraco_condorcet['Ordem Final (Condorcet)'] = ord

fraco_condorcet

,Ordem Final (Condorcet)
Efetivação do papel institucional (não reconhecimento da missão/DEAS - interna e externa),1°
"Falta de integração dos processos de trabalho (reunião de equipe, desarticulação, troca de experiências, comunicação ineficiente)",2°
Predominância de ações emergenciais em detrimento das ações estratégicas,3°
O Conselho Executivo não participa das discussões e resoluções de problemas (só informação na reunião),4°
Ausência de ferramentas gerenciais de diagnóstico e operação (indicadores),5°
Equipe reduzida,6°
Fragilidade da equipe nos conhecimentos “relacionados” ao SUS,7°
Falta de autonomia financeira interna,8°
"Ausência de integração entre pesquisa, ensino e assistência",9°
Falta organização e diretrizes para captação de recursos de emendas,10°


In [168]:
# Importing the problem data (strengths)

forte_a1 = pd.read_excel('Assets\Datasets\PO_ER460 - Strengths.xlsx', sheet_name=0)
forte_a2 = pd.read_excel('Assets\Datasets\PO_ER460 - Strengths.xlsx', sheet_name=1)
forte_a3 = pd.read_excel('Assets\Datasets\PO_ER460 - Strengths.xlsx', sheet_name=2)
forte_a4 = pd.read_excel('Assets\Datasets\PO_ER460 - Strengths.xlsx', sheet_name=3)
forte_a5 = pd.read_excel('Assets\Datasets\PO_ER460 - Strengths.xlsx', sheet_name=4)
forte_a6 = pd.read_excel('Assets\Datasets\PO_ER460 - Strengths.xlsx', sheet_name=5)
forte_a7 = pd.read_excel('Assets\Datasets\PO_ER460 - Strengths.xlsx', sheet_name=6)
forte_a8 = pd.read_excel('Assets\Datasets\PO_ER460 - Strengths.xlsx', sheet_name=7)
forte_a9 = pd.read_excel('Assets\Datasets\PO_ER460 - Strengths.xlsx', sheet_name=8)
forte_a10 = pd.read_excel('Assets\Datasets\PO_ER460 - Strengths.xlsx', sheet_name=9)
forte_a11 = pd.read_excel('Assets\Datasets\PO_ER460 - Strengths.xlsx', sheet_name=10)

In [169]:
# making the alternatives like indexes
def indice(df):
  df = df.set_index('Alternativas/Critérios')
  df.index.names = [None]
  return df

forte_a1 = indice(forte_a1)
forte_a2 = indice(forte_a2)
forte_a3 = indice(forte_a3)
forte_a4 = indice(forte_a4)
forte_a5 = indice(forte_a5)
forte_a6 = indice(forte_a6)
forte_a7 = indice(forte_a7)
forte_a8 = indice(forte_a8)
forte_a9 = indice(forte_a9)
forte_a10 = indice(forte_a10)
forte_a11 = indice(forte_a11)

In [170]:
# creating a numerical scale and passing it to df
def escala(df):
  dic = {'Muito Alto':5, 'Alto':4 , 'Médio':3, 'Baixo':2, 'Muito Baixo':1}
  df['Missão'] = df['Missão'].map(dic)
  df['Visão'] = df['Visão'].map(dic)
  return df

forte_a1 = escala(forte_a1)
forte_a2 = escala(forte_a2)
forte_a3 = escala(forte_a3)
forte_a4 = escala(forte_a4)
forte_a5 = escala(forte_a5)
forte_a6 = escala(forte_a6)
forte_a7 = escala(forte_a7)
forte_a8 = escala(forte_a8)
forte_a9 = escala(forte_a9)
forte_a10 = escala(forte_a10)
forte_a11 = escala(forte_a11)

In [171]:
# creating function to calculate preference matrix M
def M_forte(df, w):
  m_pot_oport = f_pref(df, 'Pot. Oport', 'max', 'linear', q=1, p=4)
  m_neut_ame = f_pref(df, 'Neut. Ameaça', 'max', 'linear', q=1, p=4)
  m_missao = f_pref(df, 'Missão', 'max')
  m_visao = f_pref(df, 'Visão', 'max')

  matrizes = [m_pot_oport, m_neut_ame, m_missao, m_visao]

  M = 0
  for i in range(0, len(matrizes)):
    M = M + w[i]*matrizes[i]

  return M

# vector of weights
w_forte = [0.10, 0.17, 0.25, 0.48]

# calculation of preference matrices for each decision-maker (strengths):
M_a1 = M_forte(forte_a1, w_forte)
M_a2 = M_forte(forte_a2, w_forte)
M_a3 = M_forte(forte_a3, w_forte)
M_a4 = M_forte(forte_a4, w_forte)
M_a5 = M_forte(forte_a5, w_forte)
M_a6 = M_forte(forte_a6, w_forte)
M_a7 = M_forte(forte_a7, w_forte)
M_a8 = M_forte(forte_a8, w_forte)
M_a9 = M_forte(forte_a9, w_forte)
M_a10 = M_forte(forte_a10, w_forte)
M_a11 = M_forte(forte_a11, w_forte)

C:\Users\lucas\AppData\Local\Temp\ipykernel_16368\2012082991.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  d = df[criterio][i] - df[criterio][j]


In [172]:
# calculating the net flow
def fl_forte(M):
  fl = fluxo_liq(fluxo_pos(M), fluxo_neg(M))
  return fl

fl_a1 = fl_forte(M_a1)
fl_a2 = fl_forte(M_a2)
fl_a3 = fl_forte(M_a3)
fl_a4 = fl_forte(M_a4)
fl_a5 = fl_forte(M_a5)
fl_a6 = fl_forte(M_a6)
fl_a7 = fl_forte(M_a7)
fl_a8 = fl_forte(M_a8)
fl_a9 = fl_forte(M_a9)
fl_a10 = fl_forte(M_a10)
fl_a11 = fl_forte(M_a11)

In [173]:
# function for sorting
def ordenamento(decisor, df, fl):
  # generating a dictionary for ranking
  rank = {}
  for i in range(0, len(df)):
    rank.update({df.index.values[i] : fl[i]})

  tabela = pd.DataFrame(pd.Series(rank), columns=['Fluxo Líquido'])

  tabela = tabela.sort_values(by='Fluxo Líquido', ascending=False)

  ord = []
  for i in range(0, len(df)):
    ord.append(i+1)

  tabela[f'Rank D{decisor}'] = ord

  return tabela

In [174]:
print('Avaliador 1:')
print(ordenamento(1, forte_a1, fl_a1))
print(80*'-')
print('Avaliador 2:')
print(ordenamento(2, forte_a2, fl_a2))
print(80*'-')
print('Avaliador 3:')
print(ordenamento(3, forte_a3, fl_a3))
print(80*'-')
print('Avaliador 4:')
print(ordenamento(4, forte_a4, fl_a4))
print(80*'-')
print('Avaliador 5:')
print(ordenamento(5, forte_a5, fl_a5))
print(80*'-')
print('Avaliador 6:')
print(ordenamento(6, forte_a6, fl_a6))
print(80*'-')
print('Avaliador 7:')
print(ordenamento(7, forte_a7, fl_a7))
print(80*'-')
print('Avaliador 8:')
print(ordenamento(8, forte_a8, fl_a8))
print(80*'-')
print('Avaliador 9:')
print(ordenamento(9, forte_a9, fl_a9))
print(80*'-')
print('Avaliador 10:')
print(ordenamento(10, forte_a10, fl_a10))
print(80*'-')
print('Avaliador 11:')
print(ordenamento(11, forte_a11, fl_a11))
print(80*'-')

Avaliador 1:
                                                    Fluxo Líquido  Rank D1
Equipe multiprofissional com vivências diversif...       0.576667        1
A centralização na DEAS de processos: produção ...       0.533810        2
Papel representativo e articulador da área da S...       0.471905        3
Estrutura de cargos e RH (certificação)                  0.005714        4
Área física própria                                     -0.026667        5
O Conselho Executivo facilita a integração dos ...      -0.411429        6
O modelo de Gestão Centralizada funcionou muito...      -0.486667        7
Órgãos de Saúde menores se beneficiaram com a c...      -0.663333        8
--------------------------------------------------------------------------------
Avaliador 2:
                                                    Fluxo Líquido  Rank D2
O Conselho Executivo facilita a integração dos ...       0.776190        1
Papel representativo e articulador da área da S...       0.481429   

In [175]:
# generating the tables for each decision-maker (strengths)
ordem_forte_a1 = ordenamento(1, forte_a1, fl_a1)
ordem_forte_a2 = ordenamento(2, forte_a2, fl_a2)
ordem_forte_a3 = ordenamento(3, forte_a3, fl_a3)
ordem_forte_a4 = ordenamento(4, forte_a4, fl_a4)
ordem_forte_a5 = ordenamento(5, forte_a5, fl_a5)
ordem_forte_a6 = ordenamento(6, forte_a6, fl_a6)
ordem_forte_a7 = ordenamento(7, forte_a7, fl_a7)
ordem_forte_a8 = ordenamento(8, forte_a8, fl_a8)
ordem_forte_a9 = ordenamento(9, forte_a9, fl_a9)
ordem_forte_a10 = ordenamento(10, forte_a10, fl_a10)
ordem_forte_a11 = ordenamento(11, forte_a11, fl_a11)

# creating a dataframe with the rankings of all the decision-makers:
pont_fortes = pd.concat([ordem_forte_a1, ordem_forte_a2, ordem_forte_a3, ordem_forte_a4, ordem_forte_a5, ordem_forte_a6, ordem_forte_a7, ordem_forte_a8, ordem_forte_a9, ordem_forte_a10, ordem_forte_a11], axis=1)
pont_fortes = pont_fortes.drop(['Fluxo Líquido'], axis=1)
pont_fortes

,Rank D1,Rank D2,Rank D3,Rank D4,Rank D5,Rank D6,Rank D7,Rank D8,Rank D9,Rank D10,Rank D11
"Equipe multiprofissional com vivências diversificadas, comprometida, eficiente, qualificada e com legitimidade perante à administração superior",1,5,1,1,1,4,4,4,3,3,4
"A centralização na DEAS de processos: produção de dados estatísticos, recebimento e análise das demandas dos órgãos de controle (PG, TC, MP, etc.), rotinas da execução financeira do convênio SUS e SES/SP, análise sistêmica de RH entre outros.",2,3,2,2,6,2,3,2,7,6,7
Papel representativo e articulador da área da Saúde da Unicamp com órgãos internos e externos à Universidade (maior protagonismo da área da Saúde),3,2,3,6,4,1,1,1,1,2,2
Estrutura de cargos e RH (certificação),4,7,5,7,2,5,5,5,6,4,6
Área física própria,5,6,8,8,7,6,7,6,8,8,8
O Conselho Executivo facilita a integração dos órgãos da Saúde,6,1,4,3,3,3,2,3,2,1,1
"O modelo de Gestão Centralizada funcionou muito bem durante a pandemia: aquisição ágil de EPI´s, realização de testes COVID para os servidores das unidades conveniadas, condução da distribuição reacional de vacina para comunidade",7,4,7,4,5,8,6,7,5,5,3
Órgãos de Saúde menores se beneficiaram com a captação externa de recursos pela DEAS,8,8,6,5,8,7,8,8,4,7,5


## Group Decision (Borda):

In [176]:
# creating a border scale
borda_forte = {}
for i in range(0, len(pont_fortes)):
  borda_forte.update({i+1: (len(pont_fortes)-i)})

peso_dirigente = 1.2
borda_forte_dirig = {}
for i in range(0, len(pont_fortes)):
  borda_forte_dirig.update({i+1: (len(pont_fortes)-i)*peso_dirigente})

# replacing the sort values with the Edge scale value
# Ex: (1,2,3) will become (3,2,1)
for i in pont_fortes.columns:
  if i == pont_fortes.columns[0]: # if the decision-maker is the manager (greater weight)
    pont_fortes[i] = pont_fortes[i].map(borda_forte_dirig)
  else:
    pont_fortes[i] = pont_fortes[i].map(borda_forte)

# adding up the values of each alternative
pont_fortes['Valor'] = pont_fortes.sum(axis=1)

pont_fortes_final = pd.DataFrame(pont_fortes['Valor'])
pont_fortes_final = pont_fortes_final.sort_values(by='Valor', ascending=False)

ord = []
for i in range(0, len(pont_fortes_final)):
  ord.append(f'{i+1}°')

pont_fortes_final['Ordem Final (Borda)'] = ord
pont_fortes_final

,Valor,Ordem Final (Borda)
Papel representativo e articulador da área da Saúde da Unicamp com órgãos internos e externos à Universidade (maior protagonismo da área da Saúde),74.2,1°
O Conselho Executivo facilita a integração dos órgãos da Saúde,70.6,2°
"Equipe multiprofissional com vivências diversificadas, comprometida, eficiente, qualificada e com legitimidade perante à administração superior",69.6,3°
"A centralização na DEAS de processos: produção de dados estatísticos, recebimento e análise das demandas dos órgãos de controle (PG, TC, MP, etc.), rotinas da execução financeira do convênio SUS e SES/SP, análise sistêmica de RH entre outros.",58.4,4°
Estrutura de cargos e RH (certificação),44.0,5°
"O modelo de Gestão Centralizada funcionou muito bem durante a pandemia: aquisição ágil de EPI´s, realização de testes COVID para os servidores das unidades conveniadas, condução da distribuição reacional de vacina para comunidade",38.4,6°
Órgãos de Saúde menores se beneficiaram com a captação externa de recursos pela DEAS,25.2,7°
Área física própria,22.8,8°


## Group Decision (Condorcet):

In [177]:
ordem_forte_a1 = ordenamento(1, forte_a1, fl_a1)
ordem_forte_a2 = ordenamento(2, forte_a2, fl_a2)
ordem_forte_a3 = ordenamento(3, forte_a3, fl_a3)
ordem_forte_a4 = ordenamento(4, forte_a4, fl_a4)

candidatos = ordem_forte_a1.index.values
candidatos = list(candidatos)

lista_forte = [ordem_forte_a1, ordem_forte_a2, ordem_forte_a3, ordem_forte_a4]
votos = voto(lista_forte)

# using condorcet library functions to define sorting
evaluator = condorcet.CondorcetEvaluator(candidates=candidatos, votes=votos)
winners, rest_of_table = evaluator.get_n_winners(len(ordem_forte_a1))

forte_condorcet = pd.DataFrame(index=winners)

ord = []
for i in range(0, len(forte_condorcet)):
  ord.append(f'{i+1}°')
forte_condorcet['Ordem Final (Condorcet)'] = ord

forte_condorcet

,Ordem Final (Condorcet)
"Equipe multiprofissional com vivências diversificadas, comprometida, eficiente, qualificada e com legitimidade perante à administração superior",1°
"A centralização na DEAS de processos: produção de dados estatísticos, recebimento e análise das demandas dos órgãos de controle (PG, TC, MP, etc.), rotinas da execução financeira do convênio SUS e SES/SP, análise sistêmica de RH entre outros.",2°
Papel representativo e articulador da área da Saúde da Unicamp com órgãos internos e externos à Universidade (maior protagonismo da área da Saúde),3°
O Conselho Executivo facilita a integração dos órgãos da Saúde,4°
Estrutura de cargos e RH (certificação),5°
"O modelo de Gestão Centralizada funcionou muito bem durante a pandemia: aquisição ágil de EPI´s, realização de testes COVID para os servidores das unidades conveniadas, condução da distribuição reacional de vacina para comunidade",6°
Área física própria,7°
Órgãos de Saúde menores se beneficiaram com a captação externa de recursos pela DEAS,8°
